<a href="https://colab.research.google.com/github/atilaye/Shareholding_Pattern_Data_Cleaning/blob/main/datacleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:



import numpy as np
import pandas as pd
import os

def clean_public(public_data: pd.DataFrame) -> pd.DataFrame:
    """Cleans inputted Public Shareholder Data File

    Args:
        public_data (pd.DataFrame): Public Shareholding Pattern file for a quarter
    Returns:
        pd.DataFrame: Cleaned Public Shareholding Pattern file for a quarter
    """
    # Read csv file in the function
    columns = public_data.iloc[0].tolist()
    public_data = public_data.iloc[1:].set_axis(columns, axis=1)
    if len(public_data) > 2:  # Fix: Removed parentheses around len(public_data)
        public_data = public_data.drop([1, 2])  # Remove rows index 1 and 2, not applicable when there is no row 2
    public_data = public_data.drop(public_data.columns[0], axis=1)
    public_data.rename(columns={'Total nos. shares held': 'Number of Shares Held '}, inplace=True)
    # Assuming the column was renamed, update the code accordingly
    public_data.rename(columns={'Category of shareholder': 'Category & Name of the Shareholders',
                                'Total no.s shares held': 'No. of Shares Owned',
                                'Shareholding as a % of total no. of shares (calculated as per SCRR, 1957)As a % of (A+B+C2)': '% of Shares Owned'},
                       inplace=True)  # Rename columns
    public_data['Category & Name of the Shareholders'].replace({'Sub Total A1': 'Subtotal Indian Nationals',
                                                               'Sub Total A2': 'Subtotal Foreign Nationals',
                                                               'A=A1+A2': 'Total'}, inplace=True)
    # Fill NaN values with an empty string
    public_data['Category & Name of the Shareholders'] = public_data['Category & Name of the Shareholders'].fillna('')
    # Find the index of 'A1) Indian' and 'A2) Foreign'
    indian_index = public_data[
        public_data['Category & Name of the Shareholders'].str.contains('A1.*Indian', regex=True, case=False,
                                                                       na=False)].index.tolist()
    foreign_index = public_data[
        public_data['Category & Name of the Shareholders'].str.contains('A2.*Foreign', regex=True, case=False,
                                                                       na=False)].index.tolist()
    # Find the index of the last row
    last_index = public_data.index[0]  # -1
    # Create a new column 'Nationality' with default value 'Unknown'
    public_data['Nationality'] = 'Unknown'
    # Set 'Nationality' to 'Indian' if the list is non-empty
    if indian_index:
        public_data.loc[indian_index[0]:foreign_index[0], 'Nationality'] = 'Indian'
    # Set 'Nationality' to 'Foreign' if both lists are non-empty
    if foreign_index:
        public_data.loc[foreign_index[0]:last_index, 'Nationality'] = 'Foreign'
    # Convert 'Category & Name of the Shareholders' column to string
    public_data['Category & Name of the Shareholders'] = public_data['Category & Name of the Shareholders'].astype(str)

    # Drop rows containing specific keywords in 'Category & Name of the Shareholders' column
    public_data['Category & Name of the Shareholders'] = public_data['Category & Name of the Shareholders'].fillna('')
    name_of_shareholder_keywords = 'Total|Sub Total|A1|A2|#|Individuals|Any Other|Central Government|Financial Institutions/Banks'
    public_data = public_data[~public_data['Category & Name of the Shareholders'].str.contains(name_of_shareholder_keywords, regex=True, case=False)]

    # Drop columns containing specific keywords
    column_keywords = 'equity|Voting Rights|dematerialized'  # also SubCategory (roman numerals?) and Sub Categorization, NaN

    # Filter only non-NaN values and then drop the columns
    public_data = public_data.dropna(axis=1, how='all')


    # Drop rows where all values are NaN or 0

    public_data = public_data[~(public_data == 0).all(axis=1)].reset_index(drop=True)

    # find rows w NaN and drop
    rows_with_nan = public_data[public_data.isnull().any(axis=1)]
    public_data = public_data.drop(rows_with_nan.index)



    # removing redundant Number of Shares pledged or otherwise encumbered column
    public_data.iloc[:, [5, 6]] = public_data.iloc[:, [6, 5]].to_numpy()  # to swap columns
    public_data = public_data.loc[:, ~public_data.columns.duplicated()]  # drops duplicate columns and keeps the first one

    return public_data


#test case
public_data =pd.read_csv("C:/Users/owner/Desktop/AComp/Summer 2023/algoanalytics/shareholding-data-20240103T115438Z-001/APOLLO HOSPITALS ENTERPRISE LTD/APOLLO HOSPITALS ENTERPRISE LTDJune2023Public.csv")

# Call the clean function
cleaned_dataframe = clean_public(public_data)

# Display the cleaned DataFrame
print (cleaned_dataframe.head(100))
print(public_data.iloc[:10, :5])
print(public_data.head(10).to_string())

In [ ]:


import numpy as np
import pandas as pd
import os

def clean_promoter(promoter_data: pd.DataFrame) -> pd.DataFrame:
    """Cleans inputted Promoter Shareholder Data File

    Args:
        promoter_data (pd.DataFrame): Promoter Shareholding Pattern file for a quarter
    Returns:
        pd.DataFrame: Cleaned Promoter Shareholding Pattern file for a quarter
    """
    # Read csv file in the function
    columns = promoter_data.iloc[0].tolist()
    promoter_data = promoter_data.iloc[1:].set_axis(columns, axis=1)
    if len(promoter_data) > 2:  # Fix: Removed parentheses around len(promoter_data)
        promoter_data = promoter_data.drop([1, 2])  # Remove rows index 1 and 2, not applicable when there is no row 2
    promoter_data = promoter_data.drop(promoter_data.columns[0], axis=1)
    promoter_data.rename(columns={'Total nos. shares held': 'Number of Shares Held '}, inplace=True)
    # Assuming the column was renamed, update the code accordingly
    promoter_data.rename(columns={'Category of shareholder': 'Category & Name of the Shareholders',
                                'Total no.s shares held': 'No. of Shares Owned',
                                'Shareholding as a % of total no. of shares (calculated as per SCRR, 1957)As a % of (A+B+C2)': '% of Shares Owned'},
                       inplace=True)  # Rename columns
    promoter_data['Category & Name of the Shareholders'].replace({'Sub Total A1': 'Subtotal Indian Nationals',
                                                               'Sub Total A2': 'Subtotal Foreign Nationals',
                                                               'A=A1+A2': 'Total'}, inplace=True)
    # Fill NaN values with an empty string
    promoter_data['Category & Name of the Shareholders'] = promoter_data['Category & Name of the Shareholders'].fillna('')
    # Find the index of 'A1) Indian' and 'A2) Foreign'
    indian_index = promoter_data[
        promoter_data['Category & Name of the Shareholders'].str.contains('A1.*Indian', regex=True, case=False,
                                                                       na=False)].index.tolist()
    foreign_index = promoter_data[
        promoter_data['Category & Name of the Shareholders'].str.contains('A2.*Foreign', regex=True, case=False,
                                                                       na=False)].index.tolist()
    # Find the index of the last row
    last_index = promoter_data.index[0]  # -1
    # Create a new column 'Nationality' with default value 'Unknown'
    promoter_data['Nationality'] = 'Unknown'
    # Set 'Nationality' to 'Indian' if the list is non-empty
    if indian_index:
        promoter_data.loc[indian_index[0]:foreign_index[0], 'Nationality'] = 'Indian'
    # Set 'Nationality' to 'Foreign' if both lists are non-empty
    if foreign_index:
        promoter_data.loc[foreign_index[0]:last_index, 'Nationality'] = 'Foreign'
    # Convert 'Category & Name of the Shareholders' column to string
    promoter_data['Category & Name of the Shareholders'] = promoter_data['Category & Name of the Shareholders'].astype(str)

    # Drop rows containing specific keywords in 'Category & Name of the Shareholders' column
    promoter_data['Category & Name of the Shareholders'] = promoter_data['Category & Name of the Shareholders'].fillna('')
    name_of_shareholder_keywords = 'Total|Sub Total|A1|A2|#|Individuals|Any Other|Central Government|Financial Institutions/Banks'
    promoter_data = promoter_data[~promoter_data['Category & Name of the Shareholders'].str.contains(name_of_shareholder_keywords, regex=True, case=False)]

    # Drop columns containing specific keywords
    column_keywords = 'equity|Voting Rights|dematerialized|Total nos. shares held'

    # Filter only non-NaN values and then drop the columns
    promoter_data = promoter_data.dropna(axis=1, how='all')


    # Drop rows where all values are NaN or 0

    promoter_data = promoter_data[~(promoter_data == 0).all(axis=1)].reset_index(drop=True)

    # find rows w NaN and drop
    rows_with_nan = promoter_data[promoter_data.isnull().any(axis=1)]
    promoter_data = promoter_data.drop(rows_with_nan.index)


    # removing redundant Number of Shares pledged or otherwise encumbered column
    promoter_data.iloc[:, [5, 6]] = promoter_data.iloc[:, [6, 5]].to_numpy()  # to swap columns
    promoter_data = promoter_data.loc[:, ~promoter_data.columns.duplicated()]  # drops duplicate columns and keeps the first one

    return promoter_data

#test case
promoter_data =pd.read_csv("C:/Users/owner/Desktop/AComp/Summer 2023/algoanalytics/shareholding-data-20240103T115438Z-001/APOLLO HOSPITALS ENTERPRISE LTD/APOLLO HOSPITALS ENTERPRISE LTDJune2023Public.csv")

# Call the clean function
cleaned_dataframe = clean_public(promoter_data)

# Display the cleaned DataFrame
print (cleaned_dataframe.head(100))
print(promoter_data.iloc[:10, :5])
print(promoter_data.head(10).to_string())

In [ ]:


import numpy as np
import pandas as pd
import os

def clean_summary(summary_data: pd.DataFrame) -> pd.DataFrame:
    """Cleans inputted  Shareholder Summary Data File

    Args:
        summary_data (pd.DataFrame): Shareholding Summary Pattern file for a quarter
    Returns:
        pd.DataFrame: Cleaned Shareholding Summary Pattern file for a quarter
    """
    # Read csv file in the function
    columns = summary_data.iloc[0].tolist()
    summary_data = summary_data.iloc[1:].set_axis(columns, axis=1)
    if len(summary_data) > 2:  # Fix: Removed parentheses around len(summary_data)
        summary_data = summary_data.drop([1, 2])  # Remove rows index 1 and 2, not applicable when there is no row 2
    summary_data = summary_data.drop(summary_data.columns[0], axis=1)
    summary_data.rename(columns={'Total nos. shares held': 'Number of Shares Held '}, inplace=True)
    # Assuming the column was renamed, update the code accordingly
    summary_data.rename(columns={'Category of shareholder': 'Category & Name of the Shareholders',
                                'Total no.s shares held': 'No. of Shares Owned',
                                'Shareholding as a % of total no. of shares (calculated as per SCRR, 1957)As a % of (A+B+C2)': '% of Shares Owned'},
                       inplace=True)  # Rename columns
    summary_data['Category & Name of the Shareholders'].replace({'Sub Total A1': 'Subtotal Indian Nationals',
                                                               'Sub Total A2': 'Subtotal Foreign Nationals',
                                                               'A=A1+A2': 'Total'}, inplace=True)
    # Fill NaN values with an empty string
    summary_data['Category & Name of the Shareholders'] = summary_data['Category & Name of the Shareholders'].fillna('')
    # Find the index of 'A1) Indian' and 'A2) Foreign'
    indian_index = summary_data[
        summary_data['Category & Name of the Shareholders'].str.contains('A1.*Indian', regex=True, case=False,
                                                                       na=False)].index.tolist()
    foreign_index = summary_data[
        summary_data['Category & Name of the Shareholders'].str.contains('A2.*Foreign', regex=True, case=False,
                                                                       na=False)].index.tolist()
    # Find the index of the last row
    last_index = summary_data.index[0]  # -1
    # Create a new column 'Nationality' with default value 'Unknown'
    summary_data['Nationality'] = 'Unknown'
    # Set 'Nationality' to 'Indian' if the list is non-empty
    if indian_index:
        summary_data.loc[indian_index[0]:foreign_index[0], 'Nationality'] = 'Indian'
    # Set 'Nationality' to 'Foreign' if both lists are non-empty
    if foreign_index:
        summary_data.loc[foreign_index[0]:last_index, 'Nationality'] = 'Foreign'
    # Convert 'Category & Name of the Shareholders' column to string
    summary_data['Category & Name of the Shareholders'] = summary_data['Category & Name of the Shareholders'].astype(str)

    # Drop rows containing specific keywords in 'Category & Name of the Shareholders' column
    summary_data['Category & Name of the Shareholders'] = summary_data['Category & Name of the Shareholders'].fillna('')
    name_of_shareholder_keywords = 'Total|Sub Total|A1|A2|#|Individuals|Any Other|Central Government|Financial Institutions/Banks|underlying|Employee'
    summary_data = summary_data[~summary_data['Category & Name of the Shareholders'].str.contains(name_of_shareholder_keywords, regex=True, case=False)]

    # Drop columns containing specific keywords
    column_keywords = 'equity|Voting Rights|dematerialized'

    # Filter only non-NaN values and then drop the columns

    summary_data = summary_data.dropna(axis=1, how='all')


    # Drop rows where all values are NaN or 0

    summary_data = summary_data[~(summary_data == 0).all(axis=1)].reset_index(drop=True)

    # find rows w NaN and drop
    rows_with_nan = summary_data[summary_data.isnull().any(axis=1)]
    summary_data = summary_data.drop(rows_with_nan.index)


    # removing redundant Number of Shares pledged or otherwise encumbered column
    summary_data.iloc[:, [5, 6]] = summary_data.iloc[:, [6, 5]].to_numpy()  # to swap columns
    summary_data = summary_data.loc[:, ~summary_data.columns.duplicated()]  # drops duplicate columns and keeps the first one

    return summary_data


#test case
summary_data =pd.read_csv("C:/Users/owner/Desktop/AComp/Summer 2023/algoanalytics/shareholding-data-20240103T115438Z-001/APOLLO HOSPITALS ENTERPRISE LTD/APOLLO HOSPITALS ENTERPRISE LTDJune2023Public.csv")

# Call the clean function
cleaned_dataframe = clean_public(summary_data)

# Display the cleaned DataFrame
print (cleaned_dataframe.head(100))
print(summary_data.iloc[:10, :5])
print(summary_data.head(10).to_string())